In [2]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

import tensorflow as tf
import keras
from keras.models import Sequential, load_model, Model
from keras.layers import Flatten, Dense, Activation, BatchNormalization, Conv2D, Lambda, Dropout
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
import time
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
from PIL import Image

OUTPUT_NORMALIZATION = 655.35

tf.set_random_seed(1)
np.random.seed(1)

df = pd.read_csv('data/label.csv', header=0)
df = df.sample(frac=1).reset_index(drop=True)

nb_of_training_ex = 140800
train_df = df[:nb_of_training_ex]
validation_df = df[nb_of_training_ex:]

img_shape = (66, 200, 3)

batch_size = 128

def generator(df, batch_size):

    img_list = df['img']
    wheel_axis = df['wheel-axis']
    
    # create empty batch
    batch_img = np.zeros((batch_size,) + img_shape)
    batch_label = np.zeros((batch_size, 1))
    
    index = 0
    while True:
        for i in range(batch_size):
            
            label = wheel_axis.iloc[index]
            img_name = img_list.iloc[index]
            
            pil_img = image.load_img('data/images/roi/'+img_name)
            
            if(np.random.choice(2, 1)[0] == 1):
                pil_img = pil_img.transpose(Image.FLIP_LEFT_RIGHT)
                label = -1 * label
            
            batch_img[i] = image.img_to_array(pil_img)
            batch_label[i] = label
            
            index += 1
            if index == len(img_list):
                #End of epoch, reshuffle
                df = df.sample(frac=1).reset_index(drop=True)
                img_list = df['img']
                wheel_axis = df['wheel-axis']
                
                index = 0
        yield batch_img / 255.0, (batch_label / OUTPUT_NORMALIZATION)

train_steps = (train_df.shape[0] / batch_size)
val_steps = (validation_df.shape[0] / batch_size) + 1

print("total examples set %d, training set %d, vaidation set %d" % (df.shape[0], nb_of_training_ex, validation_df.shape[0]))
print("batch_size: %d, train_steps: %d, val_steps: %d" % 
      (batch_size, train_steps, val_steps))

total examples set 162495, training set 140800, vaidation set 21695
batch_size: 128, train_steps: 1100, val_steps: 170


In [3]:
train_batch = generator(train_df, batch_size)
validation_batch = generator(validation_df, batch_size)

In [4]:
def getPilotNetModel(input_shape):
    model = Sequential([
        Conv2D(24, kernel_size=(5,5), strides=(2,2), activation='relu', input_shape=input_shape),
        BatchNormalization(),
        Conv2D(36, kernel_size=(5,5), strides=(2,2), activation='relu'),
        BatchNormalization(),
        Conv2D(48, kernel_size=(5,5), strides=(2,2), activation='relu'),
        BatchNormalization(),
        Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu'),
        BatchNormalization(),
        Conv2D(64, kernel_size=(3,3), strides=(1,1), activation='relu'),
        BatchNormalization(),
        Flatten(),
        Dense(100, activation='relu'),
        BatchNormalization(),
        Dense(50, activation='relu'),
        BatchNormalization(),
        Dense(10, activation='relu'),
        BatchNormalization(),
        Dense(1)
    ])
    return model

In [5]:
model = getPilotNetModel(img_shape)
sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss="mse", metrics=['mae', 'acc'])
print(model.summary())

model_file_name= 'drive-model' + '-{epoch:03d}-{val_loss:.5f}.h5'
callbacks_list = [
    ModelCheckpoint(model_file_name, monitor='val_mean_absolute_error', verbose=1, save_best_only=False),
    EarlyStopping(monitor='val_mean_absolute_error', patience=5, verbose=0),
    TensorBoard(log_dir='./tensorboard/', histogram_freq=0, write_graph=False, write_images=False)
    ]

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 31, 98, 24)        1824      
_________________________________________________________________
batch_normalization_1 (Batch (None, 31, 98, 24)        96        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 47, 36)        21636     
_________________________________________________________________
batch_normalization_2 (Batch (None, 14, 47, 36)        144       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 5, 22, 48)         43248     
_________________________________________________________________
batch_normalization_3 (Batch (None, 5, 22, 48)         192       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 20, 64)         27712     
__________

In [6]:
model.fit_generator(train_batch, 
                    train_steps, 
                    epochs=50, 
                    verbose=1, 
                    callbacks=callbacks_list, 
                    validation_data=validation_batch, 
                    validation_steps=val_steps,
                    initial_epoch=0)

model.save('drive-model.h5')

Epoch 1/50
1100/1100 [==============================] - 2134s 2s/step - loss: 0.9619 - mean_absolute_error: 0.6242 - acc: 0.0062 - val_loss: 0.6358 - val_mean_absolute_error: 0.4995 - val_acc: 0.0085

Epoch 00001: saving model to drive-model-001-0.63577.h5
Epoch 2/50
1100/1100 [==============================] - 2171s 2s/step - loss: 0.4937 - mean_absolute_error: 0.4687 - acc: 0.0069 - val_loss: 0.4482 - val_mean_absolute_error: 0.4321 - val_acc: 0.0083

Epoch 00002: saving model to drive-model-002-0.44822.h5
Epoch 3/50
1100/1100 [==============================] - 3271s 3s/step - loss: 0.3966 - mean_absolute_error: 0.4296 - acc: 0.0070 - val_loss: 1.3538 - val_mean_absolute_error: 0.8178 - val_acc: 0.0046

Epoch 00003: saving model to drive-model-003-1.35379.h5
Epoch 4/50
1100/1100 [==============================] - 1844s 2s/step - loss: 0.3387 - mean_absolute_error: 0.4034 - acc: 0.0072 - val_loss: 0.3564 - val_mean_absolute_error: 0.4046 - val_acc: 0.0076

Epoch 00004: saving model to

1100/1100 [==============================] - 1730s 2s/step - loss: 0.1290 - mean_absolute_error: 0.2711 - acc: 0.0075 - val_loss: 0.1276 - val_mean_absolute_error: 0.2534 - val_acc: 0.0083

Epoch 00033: saving model to drive-model-033-0.12757.h5
Epoch 34/50
1100/1100 [==============================] - 1733s 2s/step - loss: 0.1264 - mean_absolute_error: 0.2696 - acc: 0.0075 - val_loss: 0.1213 - val_mean_absolute_error: 0.2515 - val_acc: 0.0093

Epoch 00034: saving model to drive-model-034-0.12129.h5
Epoch 35/50
1100/1100 [==============================] - 1730s 2s/step - loss: 0.1260 - mean_absolute_error: 0.2685 - acc: 0.0076 - val_loss: 0.1185 - val_mean_absolute_error: 0.2458 - val_acc: 0.0091

Epoch 00035: saving model to drive-model-035-0.11854.h5
Epoch 36/50
1100/1100 [==============================] - 1730s 2s/step - loss: 0.1225 - mean_absolute_error: 0.2653 - acc: 0.0076 - val_loss: 0.1180 - val_mean_absolute_error: 0.2459 - val_acc: 0.0088

Epoch 00036: saving model to drive-m